In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [4]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [5]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# Evaluation of model DIC and TA bias matched to salinity

**Notebook overview:** We understand that our carbon model, SalishSeaCast-SKØG, has a slight bias ($\sim$+25 umol/L) in the total alkalinity field. This small bias results in a non-negligible bias in the environmentally-relevant $\Omega_A$ metric that we often consider ($\sim$+0.2). However

In [6]:
#imports
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import seawater
import netCDF4 as nc
from scipy import stats

from numpy import *
from scipy import *

import netCDF4 as nc
import numpy as np
import scipy as sp
import cmocean
import glob
import seawater
import arrow
from salishsea_tools import (
    nc_tools,
    viz_tools,
    geo_tools,
    tidetools
)


In [7]:
infil = np.loadtxt('./Datasets/grl2016_nu.txt')

crid= infil[:,0]
ln = infil[:,2]
stn = infil[:,3]
mon = infil[:,4]
day = infil[:,5]
lat_or = infil[:,6]
lon_or = infil[:,7]
lat = infil[:,6]
lon = infil[:,7]
P = infil[:,8]
T = infil[:,9]
S = infil[:,10]
ox = infil[:,11]
ox_qf = infil[:,12]
dic = infil[:,13]
alk = infil[:,15]
dic_qf = infil[:,14]
alk_qf = infil[:,16]
no3 = infil[:,17]
no3_qf = infil[:,18]
si = infil[:,19]
si_qf = infil[:, 20]
po4 = infil[:,21]
po4_qf = infil[:, 22]

##add model coords
obs_i = np.zeros_like(alk)
obs_j = np.zeros_like(alk)
moddepth = np.zeros_like(alk)
depthdist_modobs = np.zeros_like(alk)

mesh = nc.Dataset('/data/tjarniko/MEOPAR/grid/bathymetry_201702.nc')
grid = nc.Dataset('/data/tjarniko/MEOPAR/grid/mesh_mask201702.nc')
nav_lon = mesh.variables['nav_lon'][:]
nav_lat = mesh.variables['nav_lat'][:]
bathy = mesh.variables['Bathymetry'][:]
depth = grid.variables['gdept_1d'][:]

dist = []
for w in range(0,len(lat)):
    j, i = geo_tools.find_closest_model_point(lon[w],lat[w],nav_lon,nav_lat)
    obs_j[w] = j
    obs_i[w] = i
    close = np.abs(depth-P[w])
    # smallest
    depthdist_modobs[w] = (np.min(close))
    moddepth[w] = (np.argmin(close))
    

In [9]:
mask = (((dic_qf==2) | (dic_qf==6)) & \
        ((alk_qf==2) | (alk_qf==6)) & \
        (dic > 0) & (alk >0))
print(np.shape(dic[mask]))

### get accessable subset


dens = seawater.dens(S,T,P)
sal_conv = S * 35.16504/35
# CONVERT FROM UMOL TO MMOL/M3
ta_conv=alk*dens/1000
dic_conv=dic*dens/1000
sal_conv = S * 35.16504/35

### subset, converted to model units
dic_SSconv = dic_conv[mask]
ta_SSconv = ta_conv[mask]
sal_SSconv = sal_conv[mask]
depth_SSconv = P[mask]; lat_SSconv = lat[mask]; lon_SSconv = lon[mask]
mon_SSconv = mon[mask]; day_SSconv = day[mask]; i_SSconv = obs_i[mask].astype(int);
j_SSconv = (obs_j[mask].astype(int))
moddepth_SSconv = moddepth[mask]; depthdist_modobs_SSconv = depthdist_modobs[mask]


(425,)


In [14]:
def find_model_dat_bysal(w, yr):
    
    sal_marker = 1
    TA_int = np.nan
    
    #find data file 
    mon = mon_SSconv[w]; day = day_SSconv[w]
    tdate = f'{yr}-{int(mon)}-{int(day)}'
    td = arrow.get(tdate)
    ymd = td.format('YYYYMMDD')
    #print(ymd)
    
    #open the relevant carp and grid files
    ptd = f'/data/tjarniko/results/hindcast.201905_dayavg_DIC-TA-T-S/carp_1d_{ymd}.nc'
    ptd2 = glob.glob(ptd)
    carpnc = nc.Dataset(ptd2[0])
    ptd = f'/data/tjarniko/results/hindcast.201905_dayavg_DIC-TA-T-S/grid_1d_{ymd}.nc'
    ptd2 = glob.glob(ptd)
    gridnc = nc.Dataset(ptd2[0])
    
    # extract salinity, temperature, etc
    salvec = gridnc['model_output']['SAL'][:,j_SSconv[w],i_SSconv[w]]
    salvec[salvec == 0] = np.nan
    
    tempvec = gridnc['model_output']['TEMP'][:,j_SSconv[w],i_SSconv[w]]
    tempvec[tempvec == 0] = np.nan

    TAvec = carpnc['model_output']['TA'][:,j_SSconv[w],i_SSconv[w]]
    TAvec[TAvec == 0] = np.nan

    DICvec = carpnc['model_output']['DIC'][:,j_SSconv[w],i_SSconv[w]]
    DICvec[DICvec == 0] = np.nan
    

    try:
        sal_lb = (np.max(salvec[salvec<sal_SSconv[w]]))
        lb_loc = np.where(salvec == sal_lb)
        TA_lb = TAvec[lb_loc]; DIC_lb = DICvec[lb_loc]; 
    except:
        #print(f'obs fresher than all mod points here, mod min sal is:{np.nanmin(salvec)}')
        sal_marker = 2
    ### min of things higher than the obs
    try:
        sal_ub = (np.min(salvec[salvec>sal_SSconv[w]]))
        ub_loc = np.where(salvec == sal_ub)
        TA_ub = TAvec[ub_loc]
        #print(ub_loc)
    except:
        #print(f'obs saltier than all mod points here, mod max sal is:{np.nanmax(salvec)}')
        sal_marker = 3

    if sal_marker == 1:
        
        saldiff = sal_ub-sal_lb
        
        TAdiff2 = TA_ub-TA_lb
        TAdiff = TAdiff2[0]
        
        
        
        #print(TAdiff)
#         if (TA_ub-TA_lb<0):
#             print(w)
        sal_extraprop = (sal_SSconv[w] - sal_lb)/saldiff
        #print(sal_extraprop)
        
        TA_int = TA_lb + sal_extraprop * TAdiff
        
    gridnc.close(); carpnc.close()
    
    return lb_loc, ub_loc, sal_marker, TA_int
    



In [15]:
clos_S = np.zeros(425)
plclos_S = np.zeros(425)
sms = np.zeros(425)
TA_ints = np.zeros(425)

for i in range(0,1):
    if i%10 == 0: 
        print(i)
    clos_S[i], plclos_S[i], sms[i], TA_ints[i] = find_model_dat_bysal(i,2015)

0
(array([], dtype=int64),)


/home/tjarniko/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in less
/home/tjarniko/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: RuntimeWarning: invalid value encountered in greater


NameError: name 'clossal' is not defined

In [119]:
np.nanmean(TA_ints[~np.isnan(TA_ints)]) - np.nanmean(ta_SSconv[~np.isnan(TA_ints)])

12.49126532913624

In [125]:
print(np.shape(sms[sms==1]))
print(np.shape(sms[sms==2]))
print(np.shape(sms[sms==3]))

(331,)
(56,)
(38,)


In [126]:
331+56+38

425